In [1]:
# import the SimPy package 
#from simpy import initialize
#from SimPy.Simulation import *
import simpy
import random
import collections

       
       
       
       
       
       .

In [81]:
def distribucion_llegada(env,puestos,cont,i):
    while True:
        i += 1
        r = random.random()
        print("Distribucion de generacion de tipo de cliente: %5.2f"%r)
        if r <= cont["PE"]:
            ce = Customer(name = "clienteE%02d"%i)
            ia = random.uniform(cont["min"],cont["max"])
            print("IA generado para %s: %5.2f"%(ce.name,ia))
            yield env.timeout(ia) #BIEN
            print("T=%5.2f %s: EMERGENCIA - llegue"%(env.now,ce.name))
            env.process(ce.llamada_emergencia(env,puestos,cont))
        else: 
            cc = Customer(name = "clienteC%02d"%i)
            ia = random.uniform(cont["min"],cont["max"])
            print("T=%5.2f IA generado para %s: %5.2f"%(env.now,cc.name,ia))
            yield env.timeout(ia)
            print("T=%5.2f %s: COMERCIAL - llegue"%(env.now,cc.name))
            env.process(cc.llamada_comercial(env,puestos,cont))

In [75]:
class Customer():
    """Evento llegadas/llamadas de los clientes"""
    def __init__(self,name):
        self.name = name
        
    def llamada_emergencia(self,env,puestos,cont):
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        comercial = puestos[0].request(priority=0)
        emergencia = puestos[1].request(priority=0)
        atendido = yield comercial | emergencia
        
        # Me atendieron por puestos comerciales?
        if comercial in atendido:
            cont["multitasking"]+=1
            print("Cantidad de gente multitasking:",cont["multitasking"])
            wait = env.now-arrive #Tiempo de espera
            print("T=%5.2f, %s: EMERGENCIA - Me atendieron en un puesto comercial despues de esperar %5.2f"%(env.now,self.name,wait))
            ta= abs(random.normalvariate(cont["mu"],cont["sigma"]))
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            
        else:
            wait = env.now-arrive #Tiempo de espera
            #wM.observe(wait) #Monitor de espera (sumatoria de espera en cola)
            print("T=%5.2f, %s: EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar %5.2f"%(env.now,self.name,wait))
            ta= abs(random.normalvariate(cont["mu"],cont["sigma"]))
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            
            
    def llamada_comercial(self,env,puestos,cont):
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        with puestos[0].request(priority=1) as request:
            yield request
            wait = env.now-arrive #Tiempo de espera
            print("T=%5.2f, %s: COMERCIAL - Me atendieron despues de esperar %5.2f"%(env.now,self.name,wait))
            ta= abs(random.normalvariate(cont["mu"],cont["sigma"]))
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
        

Condiciones Iniciales

In [78]:
# Experimental data
#maxNumber = 10000 #clientes
NT = 0
maxTime = 14400 #segundos
#random.seed(RANDOM_SEED)
PE = 0.12 #Porcentaje Emergencia
PC = 0.88 #Porcentaje Comercial

# TA - distribucion normal
mu = 267.64 
sigma = 297.40

# IA - uniforme
IAMin = 1
IAMax = 60

# Contadores
ci = {"multitasking":0,
     "mu":mu,
     "sigma":sigma,
     "min":IAMin,
     "max":IAMax,
     "PE":PE,
     "PC":PC}

# Puestos de atencion - variables de control
print('Ingrese cantidad de puestos comerciales')
cantCom = int(input())
print('Ingrese cantidad de puestos de emergencia')
cantEmer = int(input())

Ingrese cantidad de puestos comerciales
2
Ingrese cantidad de puestos de emergencia
3


Modelo de Simulacion

In [82]:
env = simpy.Environment()
puestos = [simpy.PriorityResource(env,capacity=cantCom),
           simpy.PriorityResource(env,capacity=cantEmer)] # menor el numero priority, mayor es la prioridad
           
env.process(distribucion_llegada(env,puestos,ci,i=NT))
env.run(until=maxTime)

Distribucion de generacion de tipo de cliente:  0.16
T= 0.00 IA generado para clienteC01: 39.66
T=39.66 clienteC01: COMERCIAL - llegue
Distribucion de generacion de tipo de cliente:  0.39
T=39.66 IA generado para clienteC02: 47.46
T=39.66, clienteC01: COMERCIAL - Me atendieron despues de esperar  0.00
Tiempo de Atencion generado para clienteC01: 130.16
T=87.12 clienteC02: COMERCIAL - llegue
Distribucion de generacion de tipo de cliente:  0.29
T=87.12 IA generado para clienteC03: 43.68
T=87.12, clienteC02: COMERCIAL - Me atendieron despues de esperar  0.00
Tiempo de Atencion generado para clienteC02: 224.31
T=130.80 clienteC03: COMERCIAL - llegue
Distribucion de generacion de tipo de cliente:  0.38
T=130.80 IA generado para clienteC04: 53.00
T=169.82, clienteC03: COMERCIAL - Me atendieron despues de esperar 39.02
Tiempo de Atencion generado para clienteC03: 201.92
T=183.80 clienteC04: COMERCIAL - llegue
Distribucion de generacion de tipo de cliente:  0.04
IA generado para clienteE05:  5

Distribucion de generacion de tipo de cliente:  0.72
T=9450.98 IA generado para clienteC319: 41.82
T=9492.80 clienteC319: COMERCIAL - llegue
Distribucion de generacion de tipo de cliente:  0.08
IA generado para clienteE320: 30.70
T=9523.50 clienteE320: EMERGENCIA - llegue
Distribucion de generacion de tipo de cliente:  0.08
IA generado para clienteE321: 19.11
T=9542.61 clienteE321: EMERGENCIA - llegue
Distribucion de generacion de tipo de cliente:  0.80
T=9542.61 IA generado para clienteC322: 30.35
T=9572.96 clienteC322: COMERCIAL - llegue
Distribucion de generacion de tipo de cliente:  0.28
T=9572.96 IA generado para clienteC323:  1.44
T=9574.40 clienteC323: COMERCIAL - llegue
Distribucion de generacion de tipo de cliente:  0.02
IA generado para clienteE324:  5.64
T=9580.04 clienteE324: EMERGENCIA - llegue
Distribucion de generacion de tipo de cliente:  0.17
T=9580.04 IA generado para clienteC325: 59.37
T=9639.41 clienteC325: COMERCIAL - llegue
Distribucion de generacion de tipo de cl

Results

In [ ]:
#result = wM.count(),wM.mean() #wM.mean --> promedio de espera en cola
print "Average wait for %3d completions was %5.3f minutes."% result

# Porcentaje de espera menor a 40 segundos
# Objetio: Mayor o igual al 90%

print('Cantidad de puestos comerciales',cantCom)
print('Cantidad de puestos de emergencia',cantEmer)

In [52]:
print("%7.4f %s: EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar %6.3f"%(2.33232342424232,'maruu da best',12.234992311))

 2.3323 maruu da best: EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar 12.235
